# **Run the Simulation**

In [13]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import time

from Functions import calculate_distance_to_bars, initialize_agents, step

In [14]:
# Function to run the simulation for a given configuration
def run_simulation(config):
    # Extract parameters from the config dictionary
    num_students = config["num_students"]
    num_police = config["num_police"]
    steps = config["steps"]
    aggress_threshold = config["aggress_threshold"]
    mode = config["mode"]
    discount = config["discount"]
    graph_type = config["graph_type"]
    
    grid_height = 10  # Default grid height
    grid_width = 16   # Default grid width
    
    # Initialize agents
    students, police_officers, grid, layout_grid = initialize_agents(grid_height, grid_width, num_students, num_police, graph_type)
    
    # Track data for plotting and analysis
    fight_history = []
    avg_aggressiveness_history = []
    fight_spots_grid = np.zeros((grid_height, grid_width))  # Initialize the grid to track fight spots
    steps_between_fights_history = []
    steps_between_fights = 0

    # Start the simulation
    for step_num in range(steps):
        if step_num % 10 == 0 and discount:
            bar_discount = True  # Apply discount every 10 steps if enabled
        
        # Run one step of the simulation
        fight_counter = step(students, police_officers, mode, grid, layout_grid, aggress_threshold, fight_spots_grid)
        
        if fight_counter > 0:
            steps_between_fights = 0
        else:
            steps_between_fights += 1
        
        # Track the results
        fight_history.append(fight_counter)
        steps_between_fights_history.append(steps_between_fights)
        avg_aggressiveness = np.mean([s.aggressiveness for s in students])
        avg_aggressiveness_history.append(avg_aggressiveness)
    
    # After the simulation, calculate distances to bars
    distances = calculate_distance_to_bars(fight_spots_grid, grid_width)
    
    # Calculate the average distance to bars
    avg_distance = np.mean(distances)

    return {
        "fight_history": fight_history,
        "steps_between_fights_history": steps_between_fights_history,
        "avg_aggressiveness_history": avg_aggressiveness_history,
        "distances": distances,
        "avg_distance": avg_distance
    }


In [15]:
# Define the possible values for each parameter
num_students_range = [10, 20, 30]  # Number of students
num_police_range = [2]    # Number of police officers
steps_range = [100]          # Number of steps
aggress_threshold_range = [0.45]  # Aggressiveness threshold
mode_range = ["distributed-strategic"]  # Mode of the simulation
discount_range = [False]  # Whether to apply discount
graph_type_range = ["barabasi"]  # Type of graph

# Generate all combinations of the parameters using itertools.product
configurations = list(itertools.product(
    num_students_range,
    num_police_range,
    steps_range,
    aggress_threshold_range,
    mode_range,
    discount_range,
    graph_type_range
))

# Convert each combination into a dictionary for easier use
configuration_dicts = [
    {
        "num_students": config[0],
        "num_police": config[1],
        "steps": config[2],
        "aggress_threshold": config[3],
        "mode": config[4],
        "discount": config[5],
        "graph_type": config[6]
    }
    for config in configurations
]

In [16]:
# Experiment configurations
def run_experiments(configurations):
    
    results = []
    for config in configurations:
        print(f"Running simulation with configuration: {config}")
        result = run_simulation(config)
        results.append(result)
        
    return results

In [17]:
# Run the experiments
results = run_experiments(configuration_dicts)

Running simulation with configuration: {'num_students': 10, 'num_police': 2, 'steps': 100, 'aggress_threshold': 0.45, 'mode': 'distributed-strategic', 'discount': False, 'graph_type': 'barabasi'}
Running simulation with configuration: {'num_students': 20, 'num_police': 2, 'steps': 100, 'aggress_threshold': 0.45, 'mode': 'distributed-strategic', 'discount': False, 'graph_type': 'barabasi'}
Running simulation with configuration: {'num_students': 30, 'num_police': 2, 'steps': 100, 'aggress_threshold': 0.45, 'mode': 'distributed-strategic', 'discount': False, 'graph_type': 'barabasi'}


In [6]:
# Plotting function
def plot_results(results):
    for i, result in enumerate(results):
        # Plot fight history
        plt.figure(figsize=(10, 6))
        plt.plot(result['fight_history'], label='Fight history')
        plt.title(f"Fight History for Simulation {i + 1}")
        plt.xlabel("Time steps")
        plt.ylabel("Number of fights")
        plt.legend()
        plt.show()

        # Plot average aggressiveness
        plt.figure(figsize=(10, 6))
        plt.plot(result['avg_aggressiveness_history'], label='Avg Aggressiveness')
        plt.title(f"Average Aggressiveness for Simulation {i + 1}")
        plt.xlabel("Time steps")
        plt.ylabel("Average Aggressiveness")
        plt.legend()
        plt.show()

        # Plot distances to bars (optional, if needed)
        plt.figure(figsize=(10, 6))
        plt.hist(result['distances'], bins=30, edgecolor='black')
        plt.title(f"Distances to Bars for Simulation {i + 1}")
        plt.xlabel("Distance")
        plt.ylabel("Frequency")
        plt.show()

        # Create a DataFrame for summary table
        summary = pd.DataFrame({
            'Time step': range(len(result['fight_history'])),
            'Fight history': result['fight_history'],
            'Avg Aggressiveness': result['avg_aggressiveness_history'],
        })
        print(f"Summary for Simulation {i + 1}:")
        display(summary.head())